#Configuração do Boto3 no Amazon Bedrock

> *Este caderno deve funcionar bem com o kernel **`Data Science 3.0`** no SageMaker Studio*

---

Neste caderno de demonstração, vemos como usar o [SDK para Python `boto3`](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) para trabalhar com os modelos de base do [Amazon Bedrock] (https://aws.amazon.com/bedrock/).

---

## Pré-requisitos

Execute as células desta seção para instalar os pacotes requeridos pelos cadernos neste workshop. ⚠️ Você verá erros de dependência de pip, que podem ser ignorados com segurança. ⚠️

IGNORAR ERRO: no momento, o solucionador de dependência de pip não considera todos os pacotes instalados. Este comportamento é a origem dos conflitos de dependência a seguir.

In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet \
    langchain==0.0.309 \
    "transformers>=4.24,<5" \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    datasets

%pip install google-search-results
%pip install SQLAlchemy
%pip install flask-sqlalchemy --user

  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/4c/84/abe01852ed53db1be164ed857a833466c6e79e63d3fffeb1e456920d615d/boto3-1.29.3-py3-none-any.whl.metadata
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/96/8b/4d8198adff8de32868558d76002cf757eb811fc52c9b966160a3d1a91a4f/awscli-1.30.3-py3-none-any.whl.metadata
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/32/47/810f68f8644145323f0bbf85f9381eb0c8ad67daa17b37d0c157996a81f6/botocore-1.32.3-py3-none-any.whl.metadata
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
  Using cached s3transfer-0.7.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached docutils-0.16-py2.py3

### Reiniciar o kernel após executar a célula acima

---

## Definições comuns de parâmetros de inferência

## Aleatoriedade e diversidade

Os modelos de base são compatíveis com os seguintes parâmetros de controle de aleatoriedade e diversidade na 
resposta.

**Temperatura** – Grandes modelos de linguagem usam probabilidade para construir as palavras em uma sequência. Para qualquer 
palavra seguinte, há uma distribuição de probabilidade de opções para a próxima palavra da sequência. Quando 
você define a temperatura mais próxima de zero, o modelo tende a selecionar as palavras de maior probabilidade. Quando 
você define a temperatura mais distante de zero, o modelo pode selecionar uma palavra de menor probabilidade.

Em termos técnicos, a temperatura modula a função densidade de probabilidade para os próximos tokens 
implementando a técnica de amostragem de temperatura. Este parâmetro pode aumentar ou achatar a curva 
da função de densidade. Um valor mais baixo resulta em uma curva mais íngreme com respostas mais deterministas; um valor 
maior resulta em uma curva mais achatada com mais respostas aleatórias.

**Top K** – A temperatura define a distribuição de probabilidade de palavras potenciais e a Top K define o ponto de corte 
em que o modelo não seleciona mais as palavras. Por exemplo, se K = 50, o modelo seleciona dentre as 50 
palavras mais prováveis que poderiam ser as próximas em uma determinada sequência. Isto reduz a probabilidade de uma palavra 
incomum ser a próxima selecionada em uma sequência.
Em termos técnicos, Top K é o número de tokens de vocabulário de maior probabilidade a serem mantidos para a filtragem
da Top K – Isso limita a distribuição dos tokens prováveis, assim o modelo escolhe um dos tokens
de maior probabilidade.

**Top P*** – Top P define um ponto de corte com base na soma das probabilidades das possíveis escolhas. Se você definir 
Top P abaixo de 1.0; o modelo considerará as opções mais prováveis e ignorará as menos prováveis. Top P é 
similar a Top K, mas em vez de limitar o número de escolhas, limita as escolhas baseadas na soma de suas 
probabilidades.
Para o exemplo de prompt “Eu ouço as batidas dos cascos de”, você pode querer que o modelo forneça “cavalos”, 
“zebras” ou “unicórnios” como a próxima palavra. Se você definir a temperatura como o número máximo, sem limitar 
Top K ou Top P, você aumenta a probabilidade de obter resultados incomuns, como “unicórnios”. Se você definir 
a temperatura como 0, você aumenta a probabilidade de “cavalos”. Se você define uma temperatura alta e define Top K ou 
Top P no máximo, você aumenta a probabilidade de “cavalos” ou “zebras” e diminui a probabilidade 
de “unicórnios”.

### Comprimento

Os parâmetros a seguir controlam o comprimento da resposta gerada.

**Comprimento da resposta** – Configura o número máximo e mínimo de tokens a serem usados na resposta 
gerada.

**Penalidade de comprimento** – A penalidade de comprimento otimiza o modelo para que seja mais conciso no resultado penalizando 
respostas mais longas. A penalidade de comprimento difere do comprimento da resposta, pois o comprimento da resposta é um ponto de corte rígido 
do comprimento mínimo ou máximo da resposta.

Em termos técnicos, a penalidade de comprimento penaliza o modelo exponencialmente por respostas longas. 0.0 
significa que não há penalidade. Defina um valor menor do que 0.0 para que o modelo gere sequências mais longas ou defina um valor 
maior do que 0.0 para que ele gere sequências mais curtas.

### Repetições

Os parâmetros a seguir controlam a repetição na resposta gerada.

*Penalidade de repetição (penalidade de presença)** – Previne repetições das mesmas palavras (tokens) nas respostas. 
1.0 significa que não há penalidade. Maior do que 1.0 diminui a repetição.